# Group $\mathsf{G}_2$

The combinatorial types $\mathsf{w}$ belonging to $\mathsf{G}_2$ are those not in $\mathsf{G}_1$, and whose dual-graph is a tree. 

In [ ]:
using Oscar
using Combinatorics
pm = Polymake;

In [ ]:
cd("..")

In [ ]:
currentDir = pwd()
include(joinpath(currentDir, "src/inputData38.jl"));
include(joinpath(currentDir, "src/fileHandling.jl"));
include(joinpath(currentDir, "src/tscCoordRing.jl"));
include(joinpath(currentDir, "src/matroidalSubd.jl"));
include(joinpath(currentDir, "src/Bmaximal.jl"));
include(joinpath(currentDir, "src/simplifyIdeal.jl"));

First, we determine which combinatorial types $\mathsf{w}$ in $\mathsf{G}_2$ that satisfy the hypotheses of Proposition 6.6, i.e., all but (possibly) one of the matroids $\mathsf{Q}_v$ (for $v\in V(\Gamma(\mathsf{w}))$) is $B$-maximal.

---
**Function**: ```test_all_But_One_BMaximal(Ms::Vector{Matroid},  F::Ring, R::GradedPolynomialRing, x::fmpz_mpoly)```

*Description*: Returns ```true``` if ```exists_Basis_With_DimB_Equals_DimTSC(M,  F)``` is ```false``` for at most 1 matroid  ```M``` in ```Ms```, and ```false``` otherwise. 


In [ ]:
function test_all_But_One_BMaximal(Ms, F, R, x)
    tests = [isBMaximal(M, F, R, x) for M in Ms]
    numberTestsTrue = length([a for a in tests if a])
    
    return length(Ms) - numberTestsTrue ≤ 1
end;

The list ```G2``` contains all combinatorial types in $\mathsf{G}_2$. 

In [ ]:
G2Path = joinpath(currentDir,"groupsFinal/G2.dat")
G2 = file2SetVectors(G2Path);

In [ ]:
fails_test_all_But_One = []
passes_test_all_But_One = []

R,x = makePolyRing(3,8,QQ)

for w in G2
    subd = SubdivisionOfPoints(vDelta38[:,2:9], -w)
    Mats = subd2Matroids(subd, 3, 8)
    if test_all_But_One_BMaximal(Mats, QQ, R, x) == false
        push!(fails_test_all_But_One, w)
    else
        push!(passes_test_all_But_One, w)
    end
end

There are 4 combinatorial types that do not satisfy the hypotheses of Proposition 6.6.

In [ ]:
length(fails_test_all_But_One) == 4

For the remaining $\mathsf{w}$, we check that all but (possibly) one of the $\mathsf{Q}_{v}$ have at least 2 parallel elements. For these $\mathsf{Q}_{v}$, the thin Schubert cells $\operatorname{Gr}(\mathsf{Q}_{v})$ are smooth and irreducible, and any face morphism with source $\operatorname{Gr}(\mathsf{Q}_{v})$ is smooth and dominant with connected fibers. We use the following functions.  

---

**Function**: ```numberParallelElements(M::Matroid)```

*Description*: This function records the number of non-loop elements parallel to another element. 

*Example*: The matroid below has ```uniform_matroid(3,4)``` as its simplification.  
```
M = matroid_from_hyperplanes([[1,2,3,8,9],[1,4,5,6,8,9],[1,7,8,9],[2,3,4,5,6,8,9],[2,3,7,8,9],[4,5,6,7,8,9]], 9)
numberParallelElements(M) = 3
```
---
**Function**: ```subd2ParallelElements(Ms::Vector{Matroid})```

*Description*: Given list of matroids ```Ms```, this function returns a list, where the ```k```-th entry records the number of parallel elements of the ```k```-th matroid of ```Ms```. 

*Example*: 

---
**Function**: ```allBut1HaveGeq2ParallelElements(w::Vector{Int64})```

*Description*: The vector ```w``` must lie in $\operatorname{Dr}(3,8)$. Let ```Ms``` be the list of matroids of maximal cells in the corresponding matroidal subdivision of $\Delta(3,8)$. This function returns ```true``` if at most 1 matroid in ```Ms``` has fewer than 2 parallel elements, and ```false``` otherwise. 

*Example*: 

---

In [ ]:
function numberParallelElements(M)
    f1 = flats(M,1)
    lps = loops(M)
    GS = matroid_groundset(M)
    return length(GS) - length(f1) - length(lps)
end;

function subd2ParallelElements(Ms)
#    subd = SubdivisionOfPoints(vDelta38[:,2:9], w)
#    MatsSubd = subd2Matroids(subd, 3, 8)
    return [numberParallelElements(M) for M in Ms]
end;

function allBut1HaveGeq2ParallelElements(w)
    subd = SubdivisionOfPoints(vDelta38[:,2:9], -w)
    Ms = subd2Matroids(subd, 3, 8)
    numParallel = subd2ParallelElements(Ms)
    geq2 = [a for a in numParallel if a ≥ 2]
    return length(numParallel) - length(geq2) ≤ 1
end;

In [ ]:
print(all([allBut1HaveGeq2ParallelElements(w) for w in fails_test_all_But_One]))

## Dimension of $\operatorname{Gr}(\mathsf{w})$

Next we check that the dimensions $\operatorname{Gr}(\mathsf{w})$ are 15 for each $\mathsf{w}$ in $\mathsf{G}_2$. 


---
**Function**: ```dimLimitTree(subd::SubdivisionOfPoints, F::Ring, d::Int64, n::Int64)```

*Description*: Computes the dimension of the inverse limit of thin Schubert cells, when the diagram is a tree (assuming the hypotheses of this dimension formula hold).

*Example*: 


In [ ]:
function dimLimitTree(subd, F, d, n, R, x)
    maxMatroids, codim1Matroids = subd2CodimLeq1Matroids(subd, d, n)
        
    dimsMax = [dimTSC_Optimized(M, F, R, x) for M in maxMatroids ]; 
    dimsCodim1 = [dimTSC_Optimized(M, F, R, x) for M in codim1Matroids   ]
    return sum(dimsMax) - sum(dimsCodim1)
end

In [ ]:
R,x = makePolyRing(3,8,QQ)
dimsTrees = []
i=1
for w in G2
    subd = SubdivisionOfPoints(vDelta38[:,2:9], -w)
    dimT = dimLimitTree(subd, QQ, 3, 8, R, x)
    push!(dimsTrees, dimT)
    #print(dimT, "\n")
end

In [ ]:
all([d == 15 for d in dimsTrees])

### Example

We consider the example $\mathsf{w} = \mathsf{e}_{126} + \mathsf{e}_{234} + \mathsf{e}_{237} + 2\mathsf{e}_{238} + \mathsf{e}_{247} + \mathsf{e}_{248} + \mathsf{e}_{278} + \mathsf{e}_{347} + \mathsf{e}_{348} + \mathsf{e}_{378} + 2\mathsf{e}_{478} + \mathsf{e}_{568}$.

In [ ]:
include(joinpath(currentDir, "src/specificMatroids.jl"));
w = [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0]
subd = SubdivisionOfPoints(vDelta38[:,2:9], w)
GammaPolymake = subd.pm_subdivision.POLYHEDRAL_COMPLEX.DUAL_GRAPH
Gamma = Graphs.Graph{Graphs.Undirected}(GammaPolymake.ADJACENCY)
Ms = subd2Matroids(subd,3,8);

The dual graph ```Gamma``` is a tree:

In [ ]:
Graphs.ne(Gamma) - Graphs.nv(Gamma) + 1 == 0

Here are the matroids of the nodes of ```Gamma```.

In [ ]:
M1 = matroidU([1],[2],[6],[3,4,5,7,8])
M2 = matroid_from_hyperplanes([[2, 3, 4, 7, 8], [5, 6, 8], [1, 2, 6], [6, 7], [5, 7], [4, 6], [4, 5], [3, 6], [3, 5], [2, 5], [1, 8], [1, 7], [1, 5], [1, 4], [1, 3]], 8)
M3 = matroid_from_hyperplanes([[1, 5, 6, 8], [1, 5, 6, 7], [1, 4, 5, 6], [1, 3, 5, 6], [1, 2, 5, 6], [4, 7, 8], [2, 3, 8], [3, 7], [3, 4], [2, 7], [2, 4]], 8)
M4 = matroidU([2],[3],[8],[1,4,5,6,7])
M5 = matroidU([5],[6],[8],[1,2,3,4,7])
M6 = matroidU([4],[7],[8],[1,2,3,5,6])

all([bases(Ms[1]) == bases(M1), 
     bases(Ms[2]) == bases(M2), 
     bases(Ms[3]) == bases(M3), 
     bases(Ms[4]) == bases(M4),
     bases(Ms[5]) == bases(M5), 
     bases(Ms[6]) == bases(M6)])

The dimensions of the thin Schubert cells of the matroids of the vertices and edges.

In [ ]:
R,x = makePolyRing(3,8,QQ)

function matroidOfEdge(Ms,e, n)
    B1 = bases(Ms[e[1]])
    B2 = bases(Ms[e[2]])
    return matroid_from_bases(intersect(B1,B2), n)
end

M12 = matroidOfEdge(Ms,[1,2],8)
M23 = matroidOfEdge(Ms,[2,3],8)
M25 = matroidOfEdge(Ms,[2,5],8)
M34 = matroidOfEdge(Ms,[3,4],8)
M36 = matroidOfEdge(Ms,[3,6],8)

all([
dimTSC_Optimized(M1, QQ, R, x) == 7,
dimTSC_Optimized(M2, QQ, R, x) == 10,
dimTSC_Optimized(M3, QQ, R, x) == 9,
dimTSC_Optimized(M4, QQ, R, x) == 7,
dimTSC_Optimized(M5, QQ, R, x) == 7,
dimTSC_Optimized(M6, QQ, R, x) == 7,
        
dimTSC_Optimized(M12, QQ, R, x) == 6,
dimTSC_Optimized(M23, QQ, R, x) == 8,
dimTSC_Optimized(M25, QQ, R, x) == 6,
dimTSC_Optimized(M34, QQ, R, x) == 6,
dimTSC_Optimized(M36, QQ, R, x) == 6
        ])